In [1]:
import pandas as pd

In [7]:
data_folder = "data/"

users_df = pd.read_csv(data_folder + "users_df.csv")
items_df = pd.read_csv(data_folder + "items_df.csv")

countries = pd.read_csv(data_folder + "countries.csv")
genres = pd.read_csv(data_folder + "genres.csv")
staff = pd.read_csv(data_folder + "staff.csv")

train_part = pd.read_csv(data_folder + "train_data.csv")#, parse_dates=["datetime"])
test_part = pd.read_csv(data_folder + "test_data.csv")
test_part = test_part.groupby("user_id").agg({"movie_id": list}).reset_index()

# Напишем парсер кинопоиска, который создаст таблицу из продолжительностей фильмов

In [8]:
items = pd.DataFrame()

In [9]:
items['title'] = items_df['title']

In [10]:
items

,title
0,"Мама, я дома"
1,Три метра над уровнем неба
2,Детектив ди и тайна призрачного пламени
3,Капитан
4,Лазурный берег
...,...
7422,Ромео + Джульетта
7423,"Поймай меня, если сможешь"
7424,Молодой Годар
7425,Юлия Колерт — «Окна»


In [11]:
movies = items['title'].values

In [12]:
movies[1000:1]

array([], dtype=object)

In [13]:
results = []

In [14]:
%%time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Функция для получения продолжительности фильма (точнее строки, которая содержит продолжительность)
def get_duration(movie_title):
    search_query = f"{movie_title}"
    url = f"https://www.kinopoisk.ru/index.php?kp_query={search_query}"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # print(soup)
        # Поиск элемента с продолжительностью
        duration_element = None
        if soup.find('div', class_='element most_wanted'):
            duration_element = soup.find('div', class_='element most_wanted')
            duration_element = duration_element.find('div', class_='info')
            duration_element = duration_element.find('span', class_='gray')
            print(duration_element)
        if duration_element:
            return duration_element.text.strip()  # Возвращаем текст продолжительности
    return None

for movie in movies[7000:7427]:
    duration = get_duration(movie)
    results.append({'Название': movie, 'Продолжительность': duration})
    time.sleep(0.2)  # Задержка между запросами, чтобы избежать блокировки

# Создание DataFrame и сохранение в CSV
df = pd.DataFrame(results)
df.to_csv('movies_durations.csv', index=False)

print("Результаты сохранены в movies_durations.csv")

<span class="gray">Pet Sematary, 101 мин</span>
<span class="gray">The Pentagon Wars, 100 мин</span>
<span class="gray">I girasoli, 107 мин</span>
<span class="gray">Paranormal Activity 4, 88 мин</span>


KeyboardInterrupt: 

In [57]:
df

,Название,Продолжительность
0,"Мама, я дома",104 мин
1,Три метра над уровнем неба,"Tres metros sobre el cielo, 123 мин"
2,Детектив ди и тайна призрачного пламени,None
3,Капитан,126 мин
4,Лазурный берег,"By the Sea, 122 мин"
5,Просто Джексон,90 мин
6,Всероссийская лига юмора. Выпуск №3. Саранск,None
7,FOMENKO_ПРОГРАММЫ Серия 1,None
8,Кристофер Робин,"Christopher Robin, 104 мин"
9,Очередь,"Darısı Başımıza, 120 мин"


In [11]:
import pickle
with open('result8.pkl', 'wb') as f:
    pickle.dump(results, f)